In [16]:
import pandas as pd
import plotly.graph_objs as go
import numpy as np

In [17]:
df = pd.read_excel('../dataset.xlsx')
df['time'] = pd.to_datetime(df['time'])
df

,time,paksha,tithi,temperature,relativeHumidityPercentage,dewpoint,windspeed,soil_moisture,hinduMonthName,vikramSamvat
0,2018-01-01 00:00:00,Shukla Paksha,Chaturdashi,7.3,90,5.8,11.2,0.214,Pausha,2074
1,2018-01-01 01:00:00,Shukla Paksha,Chaturdashi,7.1,90,5.5,13.3,0.214,Pausha,2074
2,2018-01-01 02:00:00,Shukla Paksha,Chaturdashi,6.8,90,5.2,13.9,0.214,Pausha,2074
3,2018-01-01 03:00:00,Shukla Paksha,Chaturdashi,8.9,83,6.3,12.5,0.214,Pausha,2074
4,2018-01-01 04:00:00,Shukla Paksha,Chaturdashi,13.8,62,6.7,7.7,0.214,Pausha,2074
...,...,...,...,...,...,...,...,...,...,...
32923,2021-10-03 20:00:00,Krishna Paksha,Dwadashi,26.6,78,22.5,19.5,0.349,Ashwina,2078
32924,2021-10-03 21:00:00,Krishna Paksha,Dwadashi,26.4,77,22.1,20.2,0.349,Ashwina,2078
32925,2021-10-03 22:00:00,Krishna Paksha,Dwadashi,26.2,77,21.8,19.1,0.349,Ashwina,2078
32926,2021-10-03 23:00:00,Krishna Paksha,Trayodashi,25.9,77,21.6,17.6,0.349,Ashwina,2078


In [18]:
# Resampling data from hourly to daily
# set 'time' column as the index
df = df.set_index('time')

# resample hourly data to daily
df_daily = df.resample('D').agg({'temperature': 'mean','relativeHumidityPercentage': 'mean', 'dewpoint': 'mean','windspeed': 'mean', 'soil_moisture': 'mean', 'tithi': 'first', 'hinduMonthName': 'first', 'vikramSamvat': 'first'})

# reset the index to get 'time' as a column again
df_daily = df_daily.reset_index()

df_daily
df.reset_index()


,time,paksha,tithi,temperature,relativeHumidityPercentage,dewpoint,windspeed,soil_moisture,hinduMonthName,vikramSamvat
0,2018-01-01 00:00:00,Shukla Paksha,Chaturdashi,7.3,90,5.8,11.2,0.214,Pausha,2074
1,2018-01-01 01:00:00,Shukla Paksha,Chaturdashi,7.1,90,5.5,13.3,0.214,Pausha,2074
2,2018-01-01 02:00:00,Shukla Paksha,Chaturdashi,6.8,90,5.2,13.9,0.214,Pausha,2074
3,2018-01-01 03:00:00,Shukla Paksha,Chaturdashi,8.9,83,6.3,12.5,0.214,Pausha,2074
4,2018-01-01 04:00:00,Shukla Paksha,Chaturdashi,13.8,62,6.7,7.7,0.214,Pausha,2074
...,...,...,...,...,...,...,...,...,...,...
32923,2021-10-03 20:00:00,Krishna Paksha,Dwadashi,26.6,78,22.5,19.5,0.349,Ashwina,2078
32924,2021-10-03 21:00:00,Krishna Paksha,Dwadashi,26.4,77,22.1,20.2,0.349,Ashwina,2078
32925,2021-10-03 22:00:00,Krishna Paksha,Dwadashi,26.2,77,21.8,19.1,0.349,Ashwina,2078
32926,2021-10-03 23:00:00,Krishna Paksha,Trayodashi,25.9,77,21.6,17.6,0.349,Ashwina,2078


In [19]:
# Yearly Temperature comparison for each month according to english calendar
def get_month_name(month_num):
    month_names = ['', 'January', 'February', 'March', 'April', 'May', 'June', 
                   'July', 'August', 'September', 'October', 'November', 'December']
    return month_names[month_num]

# years array
years = [2018, 2019, 2020]

# months array
months = range(1, 13)

# loop through each month and create a separate figure
for month in months:
    data = []
    layout = dict(title=f"Relative Humidity Percentage for month {get_month_name(month)}")
    std_dev_arr = []
    final_std_dev = []

    for year in years:
        # filter data for the given year and month
        df_filtered = df_daily[(df_daily['time'].dt.year == year) & (df_daily['time'].dt.month == month)]
        
        std_dev = np.std(df_filtered['relativeHumidityPercentage'])
        # print(f"standard deviation for {month} {year} is {std_dev}")
        std_dev_arr.append(std_dev)

        # create a trace for the year
        trace = go.Scatter(
            x=df_filtered['time'].dt.day,
            y=df_filtered['relativeHumidityPercentage'],
            name=str(year)
        )
        
        data.append(trace)
    
    final_std_dev.append(np.average(std_dev_arr))
    fig = go.Figure(data=data, layout=layout)
    fig.show()

print(f'Average Monthly Standard Deviation in English Calendar is {np.average(final_std_dev)}')


Average Monthly Standard Deviation in English Calendar is 5.9686131272660035


In [20]:
# Grouping hindu months together
hindu_month_grouped = df_daily.groupby(['vikramSamvat', 'hinduMonthName']).agg({'temperature': 'mean','relativeHumidityPercentage': 'mean', 'dewpoint': 'mean','windspeed': 'mean', 'soil_moisture': 'mean', 'tithi': 'first', 'hinduMonthName': 'first', 'vikramSamvat': 'first'})
hindu_month_grouped

temperature  relativeHumidityPercentage  \
vikramSamvat hinduMonthName                                            
2074         Magha             14.201437                   68.306034   
             Pausha            13.147917                   72.312500   
             Phalguna          18.242222                   61.700000   
2075         Adhik Jyeshtha    33.814028                   38.956944   
             Agrahayana        16.333190                   66.423851   
             Ashadha           29.882083                   76.954167   
             Ashwina           25.335489                   66.678161   
             Bhadra            27.327361                   81.336111   
             Chaitra           24.499337                   46.208333   
             Jyeshtha          32.148276                   47.262931   
             Kartika           21.628750                   61.608333   
             Magha             14.298420                   75.333333   
             Pausa             13.590972                   67.100000   
             Phalguna          17.830278                   64.629167   
             Shravana          29.085489                   78.622126   
             Vaishakha         28.875000                   39.354167   
2076         Agrahayana        18.753750                   68.258333   
             Ashadha           32.343056                   58.473611   
             Ashwina           26.224722                   78.183333   
             Bhadra            28.852917                   82.651389   
             Chaitra           23.704167                   60.291667   
             Jyeshtha          34.313362                   34.683908   
             Kartika           24.007471                   62.015805   
             Magha             13.000287                   77.225575   
             Pausa             11.771806                   80.330556   
             Phalguna          17.733908                   71.156609   
             Shravana          28.839224                   82.629310   
             Vaishakha         31.580278                   32.027778   
2077         Agrahayana        15.334583                   65.019444   
             Ashadha           32.889368                   58.239943   
             Ashwina           27.011638                   60.840517   
             Ashwina Adhik     28.251111                   56.597222   
             Bhadra            28.656733                   82.139010   
             Chaitra           25.421839                   40.340517   
             Jyeshtha          30.796944                   46.570833   
             Kartika           19.132083                   58.191667   
             Magha             18.099138                   64.903736   
             Pausa             13.745972                   75.545833   
             Phalguna          24.132222                   48.125000   
             Shravana          30.152299                   75.175287   
             Vaishakha         27.931034                   49.757184   
2078         Ashadha           32.517960                   60.619253   
             Ashwina           27.361859                   82.246795   
             Bhadra            27.947361                   81.686111   
             Chaitra           28.771389                   30.125000   
             Jyeshtha          31.792361                   56.486111   
             Shravana          29.085920                   78.751437   
             Vaishakha         29.864943                   45.510057   

                              dewpoint  windspeed  soil_moisture        tithi  \
vikramSamvat hinduMonthName                                                     
2074         Magha            7.773851  17.169109       0.207441    Pratipada   
             Pausha           7.564583   8.943750       0.213646  Chaturdashi   
             Phalguna        10.035694  16.712083       0.195665    Pratipada   
2075         Adhik Jyes

In [21]:
# Making a df containing paksha and tithi combined
combinedDF = df
combinedDF['paksha-tithi'] = combinedDF.apply(lambda row: row['paksha'] + '-' + row['tithi'], axis=1)
# combinedDF.drop(['paksha', 'tithi'], axis=1, inplace=True)

filtered = combinedDF[(combinedDF['vikramSamvat'] == 2074) & (combinedDF['hinduMonthName'] == 'Phalguna')]
filtered_grouped = filtered.groupby('paksha-tithi', sort=False).agg({'temperature': 'mean','relativeHumidityPercentage': 'mean', 'dewpoint': 'mean','windspeed': 'mean', 'soil_moisture': 'mean', 'hinduMonthName': 'first', 'vikramSamvat': 'first'})

filtered_grouped


,temperature,relativeHumidityPercentage,dewpoint,windspeed,soil_moisture,hinduMonthName,vikramSamvat
paksha-tithi,,,,,,,
Krishna Paksha-Pratipada,18.442857,57.333333,9.376190,20.880952,0.202000,Phalguna,2074
Krishna Paksha-Dwitiya,17.509524,68.952381,11.138095,16.290476,0.201810,Phalguna,2074
Krishna Paksha-Tritiya,16.745455,67.590909,9.963636,15.813636,0.201000,Phalguna,2074
Krishna Paksha-Chaturthi,14.440909,58.409091,5.954545,20.709091,0.201000,Phalguna,2074
Krishna Paksha-Panchami,14.825000,55.083333,5.129167,16.316667,0.200042,Phalguna,2074
Krishna Paksha-Shashthi,16.508333,47.791667,5.000000,16.379167,0.200000,Phalguna,2074
Krishna Paksha-Saptami,14.958333,46.958333,3.387500,22.845833,0.199875,Phalguna,2074
Krishna Paksha-Ashtami,14.823077,56.000000,4.873077,14.988462,0.199000,Phalguna,2074
Krishna Paksha-Navami,15.257692,54.346154,5.234615,16.803846,0.198808,Phalguna,2074


In [22]:
# Yearly Temperature comparison for each month according to Hindu calendar
def get_month_name(month_num):
    hindu_month_names = [
    '', 'Chaitra', 'Vaishakha', 'Jyeshtha', 'Ashadha', 'Shravana', 'Bhadra', 
    'Ashwina', 'Kartika', 'Agrahayana', 'Pausa', 'Magha', 'Phalguna'
    ]   
    return hindu_month_names[month_num]

# years array
years = [2075, 2076, 2077]

# months array
months = range(1, 13)

# loop through each month and create a separate figure
for month in months:
    data = []
    layout = dict(title=f"Relative Humididty Percentage for month {get_month_name(month)}")
    std_dev_arr = []
    final_std_dev = []
    
    for year in years:
        # filter data for the given year and month
        filtered = combinedDF[(combinedDF['vikramSamvat'] == year) & (combinedDF['hinduMonthName'] == get_month_name(month))]
        filtered_grouped = filtered.groupby('paksha-tithi', sort=False).agg({'temperature': 'mean','relativeHumidityPercentage': 'mean', 'dewpoint': 'mean','windspeed': 'mean', 'soil_moisture': 'mean', 'hinduMonthName': 'first', 'vikramSamvat': 'first'})

        filtered_grouped = filtered_grouped.reset_index()
        # if(month == 5) :
        #     print(year, get_month_name(month))
        #     print(filtered_grouped)

        std_dev = np.std(filtered_grouped['relativeHumidityPercentage'])
        # print(f"standard deviation for {month} {year} is {std_dev}")
        std_dev_arr.append(std_dev)

        # create a trace for the year
        trace = go.Scatter(
            x=filtered_grouped['paksha-tithi'],
            y=filtered_grouped['relativeHumidityPercentage'],
            name=str(year)
        )
        
        data.append(trace)
    
    final_std_dev.append(np.average(std_dev_arr))
    fig = go.Figure(data=data, layout=layout)
    fig.show()

print(f'Average Monthly Standard Deviation in Hindu Calendar is {np.average(final_std_dev)}')


Average Monthly Standard Deviation in Hindu Calendar is 9.594917137437598
